# Algebra and Cryptography

#### By Manuella Kristeva NAKAM YOPDUP

### Exercise 1

4. Use the Eratosthene’s sieve to find the (5000000 + n)-th prime number where n is your
birthday (mmddyy). How many prime number do we have between 2^25 and 2^26?

#### Use the Eratosthene’s sieve to find the (5000000 + n)-th prime number where n is your birthday (mmddyy)

In [92]:
import numpy as np
from math import isqrt

def eratosthene_sieve(n):

    sieve = np.ones(n + 1, dtype=bool)
    sieve[0] = sieve[1] = False

    for i in range(2, isqrt(n) + 1):
        if sieve[i]:
            sieve[i * i:n + 1:i] = False

    primes = np.nonzero(sieve)[0]
    return primes

In [93]:

limit = 100000000
n_birthday = 62702
nth_prime = 5000000 + n_birthday
print("=========================================================")
print(f"Calculating primes up to {limit}...")
print("=========================================================")

primes = eratosthene_sieve(limit)

print(f"Total number of primes found: {len(primes)}")
print("=========================================================")
print(f"The {nth_prime}, prime is: {primes[nth_prime]} ")
print("=========================================================")


Calculating primes up to 100000000...
Total number of primes found: 5761455
The 5062702, prime is: 87173869 


#### How many prime number do we have between 2^25 and 2^26?

In [94]:
#Count primes between 2^25 and 2^26
primes_in_range = [p for p in primes if pow(2,25) < p < pow(2,26)]
print("=========================================================")
num_primes = len(primes_in_range)
print(f"Number of primes between 2^25 and 2^26 : {num_primes}")
print("=========================================================")


Number of primes between 2^25 and 2^26 : 1894120


### Exercice 2. 

Bob intercepts from Alice the following encrypted message:

[[83025882561049910713, 66740266984208729661];

[117087132399404660932, 44242256035307267278];

[67508282043396028407, 77559274822593376192];

[60938739831689454113, 14528504156719159785];

[5059840044561914427, 59498668430421643612];

[92232942954165956522, 105988641027327945219];

[97102226574752360229, 46166643538418294423]]

To communicate privately, Alice and Bob are using Elgamal cryptosystem. They have
choosen the cyclique group Fp where

p = 123456789987654353003

and generated by

g = 123456789.
Their public keys are the following respectively:

gA = 52808579942366933355, gB = 39318628345168608817.

Knowing that Alice and Bob agreed that: Each message consists of a single letter which is
encoded as:

A = 11, B = 12, . . . , Z = 36, space = 41,

0 = 42, . = 43, , = 44, ? = 45.

can you help Bob to decrypt the message?

#### Necessaries functions

In [60]:
import gmpy2
from sympy import factorint


# Extended Euclidean Algorithm
def extended_gcd(a, b):
    if a == 0:
        return b, 0, 1
    gcd, x1, y1 = extended_gcd(b % a, a)
    x = y1 - (b // a) * x1
    y = x1
    return gcd, x, y

# Modular Inverse
def mod_inverse(a, p):
    gcd, x, _ = extended_gcd(a, p)
    if gcd != 1:
        raise ValueError("Inverse doesn't exist")
    return x % p

# Baby-Step Giant-Step
def baby_step_giant_step(g, x, p, d):
    m = int(gmpy2.isqrt(d)) + 1
    baby_steps = {}

    # Baby steps
    current = 1
    for j in range(m):
        baby_steps[current] = j
        current = (current * g) % p

    # Giant steps
    g_m_inv = mod_inverse(pow(g, m, p), p)
    current = x
    for i in range(m):
        if current in baby_steps:
            return i * m + baby_steps[current]
        current = (current * g_m_inv) % p
        
    raise ValueError("Logarithm not found")


# Pohlig-Hellman
def pohlig_hellman(g, x, p, e, P):
    """
        g: The base.
        x: The target value.
        p: The prime factor of the order.
        e: The exponent of the prime factor in the order.
        P: The modulus.

    Returns:
        The discrete logarithm k, such that g^k = x (mod P).
    """
    h = pow(g, (P-1) // p, P) # or pow(g, p**(e-1), P)  # Order of h is p. (This is crucial!)
    a_list = []
    xi = x
    inv_g = mod_inverse(g, P)

    for i in range(e):
        # Find a_i such that h^(a_i) = xi^(N / p^(i+1)) (mod P) where N = p^e
        xi_reduced = pow(xi, (P-1) // (p**(i+1)), P) 
        ai = baby_step_giant_step(h, xi_reduced, P, p) 

        a_list.append(ai)
        xi = (xi * pow(inv_g, ai * (p**i), P)) % P  # Update xi

    # Combine the results using the formula: k = a_0 + a_1*p + a_2*p^2 + ... + a_(e-1)*p^(e-1)
    result = sum([a * (p**i) for i, a in enumerate(a_list)])
    return result % (p**e) # Return the result modulo the order p^e


# Chinese Remainder Theorem
def Chinese_Remainder_Theorem(a, b, n, m):
  #u, v, d = Extended_Euclidean(a*n, b*m)
    d, u, v = extended_gcd(n, m)
    if d == 1:
        print("x = ", (b*u*n + a*v*m)% (m*n), "mod", m*n)
        return (b*u*n + a*v*m)% (m*n), m*n
    return


# Pohlig-Hellman Generalized
def pohlig_hellman_gen(g, p, x):
    factors = factorint(p - 1)
    print(factors)
    m = [] 
    q = []
    
    for qi, k in factors.items():
        print(qi)
        print(k)
        q.append(qi ** k)
        mi = pohlig_hellman(g, x, qi, k, p)
        print(mi)
        m.append(mi)

    if len(m) == 1:
        return m[0], q[0]  # If there's only one element, return it directly

    initm = m[0]
    initq = q[0]
    
    for i in range(1, len(m)):
        result = Chinese_Remainder_Theorem(initm, m[i], initq, q[i])
        if result is None:
            raise ValueError("Chinese_Remainder_Theorem returned None, check your implementation.")
        initm, initq = result  

    return initm, initq

#### ElGamal encryption Scheme

In [61]:
# Decrypt an ElGamal  using the private key.
def elgamal_decrypt(encrypted, private_key, p):
    
    decrypted_message = []
    # If private_key is a tuple, extract the actual integer private key
    if isinstance(private_key, tuple):
        private_key = private_key[0]  # Assuming the first element is the correct key

    for y, A in encrypted:
        s = pow(y, private_key, p)
        s_inv = mod_inverse(s, p)
        m = (A * s_inv) % p
        decrypted_message.append(m)
    return decrypted_message


# Mapping for num to char for decoding
num_to_char = {
    11: 'A', 12: 'B', 13: 'C', 14: 'D', 15: 'E', 16: 'F',
    17: 'G', 18: 'H', 19: 'I', 20: 'J', 21: 'K', 22: 'L',
    23: 'M', 24: 'N', 25: 'O', 26: 'P', 27: 'Q', 28: 'R',
    29: 'S', 30: 'T', 31: 'U', 32: 'V', 33: 'W', 34: 'X',
    35: 'Y', 36: 'Z', 41: ' ', 42: "'", 43: '.', 44: ',',
    45: '?'
}

# Decode a single number into a string using the num_to_char mapping.
def decode_number(number):
    
    num_str = str(number)
    # Split the number into pairs of digits
    pairs = [int(num_str[i:i+2]) for i in range(0, len(num_str), 2)]
    # Map each pair to a character using num_to_char
    return ''.join(num_to_char.get(pair, '?') for pair in pairs)

# Decode a list of decrypted numbers into a readable message.
def decode_message(decrypted_numbers):
    
    decoded_text = ''.join(decode_number(num) for num in decrypted_numbers)
    return decoded_text


#### Application

In [103]:
# Parameters
p = 123456789987654353003
g = 123456789
g_B = 39318628345168608817


encrypted = [
    [83025882561049910713, 66740266984208729661],
    [117087132399404660932, 44242256035307267278],
    [67508282043396028407, 77559274822593376192],
    [60938739831689454113, 14528504156719159785],
    [5059840044561914427, 59498668430421643612],
    [92232942954165956522, 105988641027327945219],
    [97102226574752360229, 46166643538418294423]
]

# Find Bob's private key using Pohlig-Hellman
private_key = pohlig_hellman_gen(g, p, g_B)
print("=========================================================")
print(f"Bob's private key: {private_key}")
print("=========================================================")

# Decrypt the message
decrypted_message = elgamal_decrypt(encrypted, private_key, p)
print(f"Decrypted  message: {decrypted_message}")
print("=========================================================")

# Decode the  message to text
decoded_text = decode_message(decrypted_message)
print(f"Decoded text: {decoded_text}")
print("=========================================================")

{2: 1, 23: 1, 1907: 1, 1407364059046241: 1}
2
1
1
23
1
16
1907
1
1377
1407364059046241
1
5191
x =  39 mod 46
x =  5191 mod 87722
x =  5191 mod 123456789987654353002
Bob's private key: (5191, 123456789987654353002)
Decrypted  message: [19244117112225192941, 16191522142944411631, 22224125164116222533, 15282944412628192319, 30193215411522152315, 24302941141124131541, 16252841182531282943]
Decoded text: IN GALOIS FIELDS, FULL OF FLOWERS, PRIMITIVE ELEMENTS DANCE FOR HOURS.


### Exercice 3.

Create your own public key and private key for Elgamal cryptosystem. Your prime number
p has to verify the following:

 p = 2p1p2 + 1 where p1 and p2 are primes;
 p1 < p2 < p1^3
;

 Its number of digits is not less that 700.
Then, Set up your own ElGamal cryptosystem. Demonstrate how a message addressed to
you can be encrypted and how you can decrypt it using your private key.

#### Necessaries functions

In [6]:
import random
from sympy import factorint

def sieve_of_eratosthenes(limit):
    sieve = [True] * (limit + 1)
    sieve[0] = sieve[1] = False
    for i in range(2, int(limit ** 0.5) + 1):
        if sieve[i]:
            for j in range(i * i, limit + 1, i):
                sieve[j] = False
    return [i for i in range(limit + 1) if sieve[i]]

def miller_rabin(n, k=5):
    if n == 2 or n == 3:
        return True
    if n < 2 or n % 2 == 0:
        return False
    r, s = 0, n - 1
    while s % 2 == 0:
        r += 1
        s //= 2
    for _ in range(k):
        a = random.randrange(2, n - 1)
        x = pow(a, s, n)
        if x == 1 or x == n - 1:
            continue
        for _ in range(r - 1):
            x = (x * x) % n
            if x == n - 1:
                break
        else:
            return False
    return True

def generate_large_prime(digits):
    while True:
        n = random.randint(10**(digits-1), 10**digits - 1)
        if miller_rabin(n):
            return n

def mod_inverse(a, m):
    def extended_gcd(a, b):
        if a == 0:
            return b, 0, 1
        gcd, x1, y1 = extended_gcd(b % a, a)
        x = y1 - (b // a) * x1
        y = x1
        return gcd, x, y
    _, x, _ = extended_gcd(a, m)
    return (x % m + m) % m



#### Generation of p1, p2, p

In [10]:
# Generate p1, p2, p
p1 = generate_large_prime(349)
p2 = generate_large_prime(350)
while p1 >= p2:
    p2 = generate_large_prime(350)
p = 2 * p1 * p2 + 1
while not miller_rabin(p) or len(str(p)) < 700:
    p1 = generate_large_prime(349)
    p2 = generate_large_prime(350)
    while p1 >= p2:
        p2 = generate_large_prime(350)
    p = 2 * p1 * p2 + 1

print(f"p1 digits: {len(str(p1))}")
print(f"p2 digits: {len(str(p2))}")
print(f"p digits: {len(str(p))}")



p1 digits: 349
p2 digits: 350
p digits: 700


#### Check if the generate numbers follow the conditions

In [71]:
from sympy import factorint
factorint(p)

{1641053944392086841921453321058207950444404193817738819473219740270491539459431794790179333198029370519198253827726748664830869236760835903417691319451047305414486954875153737239972475561536800287976368891755115763871510113547756663208548832274327268127263128729961208941625385185852405053532100044643990538296382247743234839833431437598837803711535954141156563558276004799497156738553666297843661349842435716824481757329925850674456792218265058027918615626063402333521893362715365180009430433836026547263073941949585226085934525247984227001466760694409050266285818147599806051959885649489654955078463497446671054793009320004149656371232004170389111728473163917246760060531775088682357931537722739407: 1}

In [72]:
factorint(p1)

{8836229399223340013095647802612004929452496763270039906112688880393042660857048768180580202708989666003495215047944282082281151585255618828221042735510261671384102412848721487446438793136909595200642190730921610911199593203546138400567354121452180148168251884619091701775887271604413598850098806328635477629513139087502706762929259830544462377995901: 1}

In [73]:
factorint(p2)

{92859401349195800914419340000027009681568723801475793017015602958169754059993011392652944281186250974103044906776823054555505968029620083486871686062658793692019452353380264379257810627455061341286362327958758493917072386515923960302537595419398183346193327838222062789049727171559396156231251823251982706595211633733555873147915114928658181413582003: 1}

In [74]:
p1 < p2

True

In [75]:
p1**3 > p2

True

#### Function to find a generator

In [76]:
import random

def generator(p, fact):
    while True:
        g = random.randint(2, p - 1)  # Choose a random candidate g
        
        # Check if g^(p-1)/f is congruent to 1 mod p for all factors f
        if not all(pow(g, (p - 1) // f, p) == 1 for f in fact):
            return g  # Found a valid generator



In [77]:
# Example usage
fact = [2, p1, p2]  

g = generator(p, fact)
print("Generator:", g)

Generator: 767061522441277867927379666214430753504105432228455099770725788855657397532411572780390024514944712235517146836454822966780984619405046140354959599418685404144846523924865681746729697291349862598685793451443849843025097572029847048222151924106448137656637521172078735262096629975346940122522430359424109425570898097719349621850149569306618758399345676477917566855734805753666263895463090381909922194112224101701607125849348349032565275111706054335629821998697466009038118624902864535654482400852179343310139128463017577788006886048250936721504199977114269096042281385863346198106001031938656947268542109080644311680773676993064989960073773045251892956603147664587693084386437070384596688435218556707


#### Generation of the secret key

In [78]:
d= p-1
b = random.randint(10**698, 10**699 - 1)
print(b)
B = pow(g, b, p)
B

223118762401178665456392768598989816656310577582360802552941574398929233821716474059621058203139594814878098858471442861630654818610016833895557387470536328752048428793849311103567985931485643719157611796851106106896412857626628609476743410658598603385594089586054287281966437050115776484985090499855866005887799959180126745678061422569153723347110140563601920697673928676857439967448788090273913466319134122457816825724356747271557441510102819942708139429656166820203370665152151912856409493951371270759552146575806463645678448605808317130058593359214774708345260698719395335590356935732017049403288095267803333534616052429407822013368129810942217545970524562944940524192783279849206930475544260769


532768115406393888973113375800487237673799863304661109589082899234994575751879520676339996415605433650870641751800954667979021806573434243285229059238891116713812189387886019355667238312070290149962597037046612790741181546953033147579225787523478253237947810764335401229720546141033006036105223329534587916772140866997252569465700598888102647957829140001485048078990181318548054627499281909201551316655527846518589878242464621712763122552334386109571797233946721151232088696009409859700377584817080984106165001670643550137361091146734942879363521628147365187365995431341766329433631080284337346337618387176544866908262010402373813274266020422824213133652703300339979930311709439886030150827802310735

#### Necessaries functions

In [100]:
# Extended Euclidean Algorithm
def extended_gcd(a, b):
    if a == 0:
        return b, 0, 1
    gcd, x1, y1 = extended_gcd(b % a, a)
    x = y1 - (b // a) * x1
    y = x1
    return gcd, x, y

# Modular Inverse
def mod_inverse(a, p):
    gcd, x, _ = extended_gcd(a, p)
    if gcd != 1:
        raise ValueError("Inverse doesn't exist")
    return x % p

In [99]:
import random

def letter_to_number(letter):
    
    
    letter = letter.upper()  # Ensure case-insensitivity

    if 'A' <= letter <= 'Z':
        return ord(letter) - ord('A') + 11
    elif letter == ' ':
        return 41
    elif '0' <= letter <= '9':
        return ord(letter) - ord('0') + 42  # 0 starts at 42
    elif letter == '.':
        return 43
    elif letter == ',':
        return 44
    elif letter == '?':
        return 45
    else:
        return None  # Indicate an invalid letter


def number_to_letter(number):
    """
    Transforms a number back to its corresponding letter.

    Args:
        number: The number to decode.

    Returns:
        The letter or symbol corresponding to the number, or None if the number is invalid.
    """

    if 11 <= number <= 36:
        return chr(number - 11 + ord('A'))
    elif number == 41:
        return ' '
    elif 42 <= number <= 51:
        return str(number - 42)
    elif number == 43:
        return '.'
    elif number == 44:
        return ','
    elif number == 45:
        return '?'
    else:
        return None

def encrypt_message(message, g, p, B):
    """Encrypts a message using the provided parameters and letter-to-number encoding."""
    encrypted = []
    for letter in message:
        x = letter_to_number(letter)
        if x is None:
            print(f"Encryption Error: Invalid character in message: {letter}")
            return None # Or raise an exception if desired

        a = random.randint(10**698, 10**699 - 1)
        A = pow(g, a, p)
        k = pow(B, a, p)
        y = (x * k) % p
        encrypted.append((y, A))
        #print(f"Encrypt {letter} ({x}): (y, A) = ({y}, {A})")  #Added {letter} and ({x}) to the output

    return encrypted


def decrypt_message(encrypted, b, p):
    """Decrypts a ciphertext using the private key b and modulus p."""
    decrypted = []
    for y, A in encrypted:
        k = pow(A, b, p)
        k_inv = mod_inverse(k, p)
        z = (y * k_inv) % p
        decrypted.append(z)
        print(f"Decrypt (y, A) = ({y}, {A}): z = {z}") # No change to the existing print.

    return decrypted

def decode_numbers_to_message(numbers):
    
    """
  Decodes a list of numbers back into a message string.

  Args:
    numbers: A list of integers representing the encoded message.

  Returns:
    The decoded message string.
  """
    message_decoded = ''
    for z in numbers:
        letter = number_to_letter(z)
        if letter is None:
            print(f"Decoding Error: Invalid number during decoding: {z}")
            return None  #Abort decoding if an invalid number is found.
        message_decoded += letter
    return message_decoded



#### Application

In [101]:

# Example usage:
message = "Bobs public key calculated from g b and p"

# Encryption
message = message.upper()
encrypted = encrypt_message(message, g, p, B)

if encrypted: # Only decrypt if encryption was successful
    print("=========================================================")
    # Decryption
    decrypted = decrypt_message(encrypted, b, p)
    print("=========================================================")
    # Decoding
    message_decoded = decode_numbers_to_message(decrypted)
    print("=========================================================")
    if message_decoded:
        print(f"Original: {message}")
        print("=========================================================")
        print(f"Decrypted: {message_decoded}")
        print("=========================================================")
    else:
        print("Decoding failed.")
else:
    print("Encryption failed.")

Decrypt (y, A) = (422030634743026632064415375276850908699327891034785434370279417934734644352968053806052173835058616041279461508122028858934012264086669400047498791503460754924056343757088242642282431090990634127083667227711604484547477219962132716061149493757849128278211288905791048371148825297512942971971604590925693093440032043858995631088978341941978168960057437284022427510845907277697901678382236834594548082152551672600088845736275310203532854489881831570076375302374510626368770409502638549829074367548536217386466237324693039211181477807599353405923114993723160628844780480005993407135648405932324333641649141925731755168630266255255994528584323034714471558699689289616330087714721276552780536405490805017, 70470676471488578935610686190608776393749355005529595087352815091262581779701384277280820279112095783850419220027836864336540877741739527287178308568702547390315274400460110748166699937895338886572154337023676752312449301319369393953909906585064400697645093643843137374727274825122

Decrypt (y, A) = (860398390251956934393133350777249658740290800430354190171488633111433767986924685215141201987498380390654377209051209142739925635954670434335863431208283673278509046678712888753247282531483737844267224473424380590240439404007691889103848403580987195744772283786290608668966165136213242716934363448521534791743836478673674283131165942368048066579631857792291113185534013097181274455370744748816186531498848169105813221954149244644633639660380914483741564516395315206398624418022918052414314552760592183956950907157418560992048304818747340559733047279975658185784489880705604851747807334838162663407658403001654972504834204663955784233320376312850512611490868793908140848690974273968256090992869998632, 11735317134525742886005659848287588456195153919489500476108735591504463260468323335273215932624126762218300155362190227117014483694849979313596862515681184526282245317658977730783212706862587839407623035665725501886923399029874752317675531300994951021060146157347702107283422369233

Decrypt (y, A) = (562778843264158307421215972568029072816103085528733464542618663803684768424784462571031100269013281104017333233699789151886130901725538989073226183515346510589001327109765308514036680214248284467593008719370391469530074813768445131593825609162569732840529372615109621737161322943355508701863613745496604887418712217984080177403537198998778210318897585463240074070081624782098077312583526691255070214818964642401650967171257864295605835218988512973829565096740221935212671734778902801513014639992679234446206570362127582115704394307616313866987596648016789319060616834735894264663312166235275083717115967067187147719804795317119467117076494882688632392639842412840796548706734790492679120729146629597, 15728579241440834024570941433732918507886144605098526925720265762459698920138490976776148796698735857494275717008278093881273695199339969463581277187662103653122705573715684193763169578943859742716142149649586247140566766596520572788107046883739095460535578121045836848981483374961

Decrypt (y, A) = (568996242865484331669267689553358081716480614751145401008127681768124823629082540059403976412604844552773545701453951266749505821088328844233706236364027105591900076504750478841320911788199368831862803726557602830121301636441437915141460419990348575333612472892385349184375293842462939628831944760269480439738025151847321145304368143768704101205096199833907383707986069031073152968818997152307299183861212442908066525513755168250312790308197968627583281746436325989154089362093625224666387709803554867529944192902418180240701786546751094109111930252091757297109952055829579339955213510631938179502302802401421518689425645942181670990414679753836538067596255080040787384907951271493019806218887587087, 10692224581420467157367010061224723040794258543953168492002821171358922120841989043381347839502884385777323854303734886122862379067581637587447839385377107377470573784187113649570977212444699807279588409770543157953825049511815063926746854854151349142123653848600160995293663049896

Decrypt (y, A) = (1629119210801220584644006872870910760810911264723913811416316273374435391828084661872307536772021991334163866112222968681155089143468982825869164857124524612203812496153552755689453533520699557895816791531252741540383029034769920444372574763827359693601671689129546171796772561706356735426105335882305659306745895649231116727788686447236270844251590166961199402058390251519743485239243104956172067258456412623160945739831610141997783589588966778375487826342540364792953289853784744474370642272709805806162290052042429715965603960779784713596605103531351883509675762346198252025374263125753561396944222498186917690981303625041083224043961787448735174937670750855799287589533263753342877973260737787326, 1020917346958133393475350362636776428953099913307115382858823944531016834636517849643357442409661418300349885371586167361890221033481961570115797755371374579590282304353146640374610640860080622201386501839517832881091372173141073331346652151413025590471239979142774521363813791261

In [102]:
print("=========================================================")
print(f"Original: {message}")
print("=========================================================")
print(f"Decrypted: {message_decoded}")
print("=========================================================")

Original: BOBS PUBLIC KEY CALCULATED FROM G B AND P
Decrypted: BOBS PUBLIC KEY CALCULATED FROM G B AND P
